In [ ]:
# Importamos las librerías necesarias

import pandas as pd
import re
import sqlite3

# Faker es la librería que vamos a usar para crear datos inventados

from faker import Faker
from faker.providers import BaseProvider

In [ ]:
# Creamos la conexión y el cursor

cnx = sqlite3.connect("base_datos_proveedores.db")
cursor_prov = cnx.cursor()

In [ ]:
# Inicializamos Faker en español

fake = Faker("es_ES")

In [ ]:
# Creamos la clase proveedor, a partir de cuyos atributos crearemos la tabla

class Proveedor:
    id = None
    nombre = fake.name()
    direccion = fake.address()
    provincia = re.match(patron, direccion).group(1)
    ciudad = provincia

In [ ]:
# Esto es un intento de crear direcciones con ciudad y provincia distintas
# sin tener que hacerlo manualmente
# Nos dimos cuenta de que "fake.city()" a veces devolvía nombres de provincias
# por eso la desechamos

from geopy.geocoders import Nominatim #Librería para encontrar la ubicación de una ciudad a partir de su nombre
geolocator = Nominatim(user_agent="bootcamp1") # Inicializamos el geolocalizador
dict_proveedores = {} # Inicializamos el diccionario en el que vamos a guardar nuestros proveedores
for i in range (1,21):
    dict_proveedores[f"Proveedor_{i}"] = Proveedor()
    dict_proveedores[f"Proveedor_{i}"].nombre = fake.name() # Nombre del proveedor
    dict_proveedores[f"Proveedor_{i}"].direccion = fake.street_address() # Dirección que incluye solo calle y número
    dict_proveedores[f"Proveedor_{i}"].ciudad = fake.city() # Ciudad
    info_provincia = geolocator.geocode(f"{dict_proveedores[f"Proveedor_{i}"].ciudad}").address.split(",") # Localiza la ciudad y devuelve más datos como comunidad autónoma y país, lo pasamos a una lista para quedarnos con el elemento que nos hace falta
    # Ahora vamos a seleccionar el elemento que nos interesa de la lista que hemos creado
    if len(info_provincia)>= 3:
        dict_proveedores[f"Proveedor_{i}"].provincia = info_provincia[-3] 
    else:
        dict_proveedores[f"Proveedor_{i}"].provincia = info_provincia[-2]
    
    #Asignamos una id a los proveedores
    if i < 10:
        dict_proveedores[f"Proveedor_{i}"].id = int(f"10{i}")
    else:
        dict_proveedores[f"Proveedor_{i}"].id = int(f"1{i}")

In [ ]:
# Creamos una expresión regular para buscar la ciudad en la dirección inventada
# Realmente en la dirección inventada viene la provincia, pero muchas veces coincide con el nombre de la ciudad
# Cuando este no sea el caso lo cambiaremos manualmente

patron = r".*\n(.*),"

In [ ]:
# El método que de verdad hemos usado

dict_proveedores = {}
for i in range (1,21):
    dict_proveedores[f"Proveedor_{i}"] = Proveedor()
    dict_proveedores[f"Proveedor_{i}"].nombre = fake.name()
    dict_proveedores[f"Proveedor_{i}"].direccion = fake.address()
    dict_proveedores[f"Proveedor_{i}"].ciudad = re.match(patron, dict_proveedores[f"Proveedor_{i}"].direccion).group(1) # Con el patrón creado incontramos la provincia, de momento será también nuestra ciudad
    dict_proveedores[f"Proveedor_{i}"].provincia = dict_proveedores[f"Proveedor_{i}"].ciudad
    if i < 10:
        dict_proveedores[f"Proveedor_{i}"].id = int(f"10{i}")
    else:
        dict_proveedores[f"Proveedor_{i}"].id = int(f"1{i}")

In [ ]:
# Comprobamos que ha funcionado y nos lo ha creado bien
for clave,valor in dict_proveedores.items():
    print(f"{clave}: {valor}")
    print(valor.direccion, valor.ciudad, valor.provincia)

In [ ]:
for clave, valor in dict_proveedores.items():
    query = f''' 
    INSERT INTO proveedores (id_proveedor, nombre, direccion, ciudad, provincia)
    VALUES ("{valor.id}", "{valor.nombre}", "{valor.direccion.replace("\n", ", ")}", "{valor.ciudad}", "{valor.provincia}")
    '''
    cursor_prov.execute(query)

In [ ]:
query = ''' 
SELECT *
FROM proveedores
'''
pd.read_sql(query, cnx)

In [ ]:
cnx.commit()